In [ ]:
#You can mount your Drive in Google Colab.
#Remember it will use Drive Storrage.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Run this to install ffmpeg and yt-dlp in Colab.
!apt install ffmpeg
!pip install yt-dlp

In [ ]:
import os
import subprocess
import sys
import re

def check_ytdlp_installed():
    """Check if yt-dlp is installed."""
    try:
        subprocess.run(['yt-dlp', '--version'], 
                       stdout=subprocess.PIPE, 
                       stderr=subprocess.PIPE, 
                       check=True)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False


def get_playlist_name(playlist_url):
    """
    Extract playlist name from yt-dlp to use as folder name.
    (Kept for reference, but currently unused in this version)
    """
    try:
        result = subprocess.run(
            ['yt-dlp', '--get-filename', '-o', '%(playlist_title)s', '--playlist-items', '1', playlist_url],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if result.returncode == 0:
            playlist_name = result.stdout.strip()
            playlist_name = re.sub(r'[<>:"/\\|?*]', '', playlist_name)
            playlist_name = playlist_name.strip()
            
            if playlist_name:
                return playlist_name
    except:
        pass
    
    return "videos"


def download_playlist_simple(playlist_url, base_dir=None):
    """
    Download playlist using yt-dlp's built-in templating for automatic renaming.
    Creates a 'videos' folder automatically.
    
    Args:
        playlist_url: URL of the YouTube playlist
        base_dir: Base directory where 'videos' folder will be created (default: current directory)
    """
    
    # Set default base directory
    if base_dir is None:
        base_dir = os.getcwd()
    
    playlist_folder_name = "videos"
    
    # Create 'videos' folder inside the base directory
    videos_dir = os.path.join(base_dir, playlist_folder_name)
    os.makedirs(videos_dir, exist_ok=True)
    
    print("="*80)
    print("YOUTUBE PLAYLIST DOWNLOADER - SIMPLE METHOD")
    print("="*80)
    print(f"\nVideos will be saved to: {videos_dir}")
    print(f"🔗 Playlist URL: {playlist_url}\n")
    
    # The magic template: Tut_1 - VideoTitle.mp4
    # Format: "Tut_NUMBER - TITLE.ext"
    output_template = os.path.join(videos_dir, "Tut_%(playlist_index)s - %(title)s.%(ext)s")
    
    print("Starting download...\n")
    print("-"*80)
    
    try:
        # Run yt-dlp with the template and progress display
        process = subprocess.Popen(
            [
                'yt-dlp',
                '-o', output_template,
                '--windows-filenames',   # Safe for Windows (removes special chars but keeps spaces)
                '--newline',             # Each progress line on new line (better for parsing)
                playlist_url
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )
        
        current_video = None
        total_videos = None
        
        # Read output line by line
        for line in process.stdout:
            line = line.strip()
            
            # Detect which video is being downloaded
            if '[download] Downloading item' in line:
                match = re.search(r'Downloading item (\d+) of (\d+)', line)
                if match:
                    current_video = match.group(1)
                    total_videos = match.group(2)
                    print(f"\n[{current_video}/{total_videos}] Downloading video {current_video}...")
            
            # Detect video title/destination
            elif '[download] Destination:' in line:
                # Extract filename from path
                dest_match = re.search(r'Destination: (.+)$', line)
                if dest_match:
                    full_path = dest_match.group(1)
                    filename = os.path.basename(full_path)
                    print(f"Filename: {filename}")
            
            # Show download progress
            elif '[download]' in line and '%' in line:
                # Extract progress percentage
                progress_match = re.search(r'(\d+\.\d+)%', line)
                if progress_match:
                    percentage = progress_match.group(1)
                    # Show progress on same line
                    print(f"\rProgress: {percentage}% ", end='', flush=True)
            
            # Video already downloaded
            elif 'has already been downloaded' in line:
                print("✓ Already downloaded (skipping)")
            
            # Download complete for this video
            elif '[download] 100%' in line or 'has already been downloaded' in line:
                print("\rComplete!                     ")
            
            # Show other important messages
            elif '[info]' in line or 'ERROR' in line or 'WARNING' in line:
                print(line)
        
        # Wait for process to complete
        return_code = process.wait()
        
        if return_code != 0:
            print(f"\nError: yt-dlp exited with code {return_code}")
            sys.exit(1)
        
        print("\n" + "-"*80)
        print("\nDownload complete!")
        print(f"All videos saved to: {videos_dir}")
        
        # Show downloaded files
        files = sorted([f for f in os.listdir(videos_dir) 
                       if f.startswith('Tut_')])
        
        if files:
            print(f"\nDownloaded {len(files)} videos:")
            for f in files:
                print(f"   • {f}")
        
    except FileNotFoundError:
        print(f"\nError: yt-dlp not found in system PATH")
        sys.exit(1)
    except KeyboardInterrupt:
        print("\n\nDownload interrupted by user.")
        process.terminate()
        sys.exit(1)
    except Exception as e:
        print(f"\nUnexpected error: {e}")
        sys.exit(1)


def main():
    """Main function."""
    
    print("\n" + "="*80)
    print("YOUTUBE PLAYLIST DOWNLOADER & AUTO RENAMER")
    print("="*80 + "\n")
    
    # Check if yt-dlp is installed
    if not check_ytdlp_installed():
        print("Error: yt-dlp is not installed!")
        print("\nTo install yt-dlp:")
        print("\n   Windows/Mac/Linux:")
        print("   pip install yt-dlp")
        print("\n   OR update if already installed:")
        print("   pip install -U yt-dlp")
        print("\n   OR using pipx:")
        print("   pipx install yt-dlp")
        sys.exit(1)
    
    print("yt-dlp is installed\n")
    
    # Get playlist URL
    playlist_url = input("🔗 Paste YouTube playlist URL here: ").strip()
    
    if not playlist_url:
        print("No URL provided!")
        sys.exit(1)
    
    # Get base directory (optional)
    print("\nWhere should the 'videos' folder be created?")
    print("   1. Current directory (default)")
    print("   2. Custom path")
    
    choice = input("\nChoose option (1-2) [default: 1]: ").strip()
    
    if choice == '2':
        custom_path = input("Enter custom path: ").strip().strip('"').strip("'")
        base_dir = custom_path if custom_path else os.getcwd()
    else:  # default is 1
        base_dir = os.getcwd()
    
    # Confirm
    print(f"\nA 'videos' folder will be created in: {base_dir}")
    confirm = input("Continue? (yes/no) [yes]: ").strip().lower()
    
    if confirm and confirm not in ['yes', 'y', '']:
        print("Cancelled.")
        sys.exit(0)
    
    # Download
    download_playlist_simple(playlist_url, base_dir)
    
    print("\nAll done! Enjoy your videos!")


if __name__ == "__main__":
    main()
    